In [32]:
#importing libraries:

import numpy as np
import cv2
from PIL import Image as im
import glob
import os 
from os import listdir
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
import keras
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MinMaxScaler


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator



In [42]:
src_path_train = "../data/train_frame/"
src_path_test = "../data/frames_test/"

In [81]:
# data normalization
scaler = MinMaxScaler()

image_gray = []
def data_preprocessing(image):

    img=img_to_array(image)
    img=resize(img,(227,227,3))
    gray_img=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]
    gray_img_scaled = scaler.fit_transform(gray_img)
    image_gray.append(gray_img_scaled)
    return image_gray
    


    
    
  

In [83]:

# get the path/directory
folder_dir = src_path_train
for images in os.listdir(folder_dir):
    img = Image.open('../data/train_frame/' + images)
    data_preprocessing(img)
    
 

In [84]:
gray_npimg = np.array(image_gray)
print(gray_npimg.shape)

(621, 227, 227)


In [85]:
a, b, c = gray_npimg.shape
gray_npimg.resize(b,c,a)

In [86]:
gray_npimg.shape

(227, 227, 621)

In [87]:
gray_npimg=np.clip(gray_npimg,0,1)

In [88]:
np.save('../data/trainer.npy',gray_npimg)

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential

''' The following load_model function code has been taken from 
Abnormal Event Detection in Videos using Spatiotemporal Autoencoder
by Yong Shean Chong Yong Haur Tay
Lee Kong Chian Faculty of Engineering Science, Universiti Tunku Abdul Rahman, 43000 Kajang, Malaysia.
It's main purpose is to help us generate the anomaly detector model
'''

#load_model starts here :----------------------------------------------------
def load_model():
	"""
	Return the model used for abnormal event 
	detection in videos using spatiotemporal autoencoder

	"""
	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model

In [91]:
X_train=np.load('../data/trainer.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10 to ease the load_model
#print(frames)

frames=frames-frames%10
print(frames)
X_train=X_train[:,:,:frames]
print(X_train.shape)
X_train=X_train.reshape(-1,227,227,10) # converting images to a batch of 10 each
print(X_train.shape)
#X_train = X_train[10,:,:,1]
X_train=np.expand_dims(X_train,axis=4)
print(X_train.shape)
Y_train=X_train.copy()

620
(227, 227, 620)
(62, 227, 227, 10)
(62, 227, 227, 10, 1)


In [95]:
print(Y_train[1,:,:,1,:])

[[[0.27511016]
  [0.20339164]
  [0.42501727]
  ...
  [0.49090242]
  [0.5030333 ]
  [0.4627267 ]]

 [[0.42957643]
  [0.2865811 ]
  [0.15984336]
  ...
  [0.16869505]
  [0.11932676]
  [0.1744101 ]]

 [[0.23117235]
  [0.24744841]
  [0.23412542]
  ...
  [0.36582667]
  [1.        ]
  [0.00162125]]

 ...

 [[0.4551921 ]
  [0.5016031 ]
  [0.32922876]
  ...
  [0.5498223 ]
  [0.41510442]
  [0.3127632 ]]

 [[0.52939016]
  [0.43756223]
  [0.5250708 ]
  ...
  [0.4870397 ]
  [0.36623883]
  [0.41689667]]

 [[0.5230012 ]
  [0.4692217 ]
  [0.51173806]
  ...
  [0.01490964]
  [0.46494263]
  [0.05795002]]]


In [7]:
epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  validation_split=0.1,
			  verbose=1,
			  callbacks = [callback_save]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')

Trainer has been loaded
Epoch 1/200
55/55 [==============================] - 26s 347ms/step - loss: 0.1023 - accuracy: 0.0045 - val_loss: 0.0964 - val_accuracy: 0.0045
Epoch 2/200
55/55 [==============================] - 19s 339ms/step - loss: 0.0712 - accuracy: 0.0053 - val_loss: 0.0520 - val_accuracy: 0.0064
Epoch 3/200
55/55 [==============================] - 22s 399ms/step - loss: 0.0420 - accuracy: 0.0064 - val_loss: 0.0449 - val_accuracy: 0.0064
Epoch 4/200
55/55 [==============================] - 18s 335ms/step - loss: 0.0383 - accuracy: 0.0065 - val_loss: 0.0428 - val_accuracy: 0.0065
Epoch 5/200
55/55 [==============================] - 22s 390ms/step - loss: 0.0368 - accuracy: 0.0065 - val_loss: 0.0431 - val_accuracy: 0.0065
Epoch 6/200
55/55 [==============================] - 20s 353ms/step - loss: 0.0360 - accuracy: 0.0066 - val_loss: 0.0428 - val_accuracy: 0.0065
Epoch 7/200
55/55 [==============================] - 19s 347ms/step - loss: 0.0354 - accuracy: 0.0066 - val_loss